In [1]:
from osgeo import ogr, gdal, gdal_array
import numpy as np

In [2]:
source_path = 'E:\\ML\\satellite-image-analysis\\satellite-image-analysis-project\\sources' #rel path: 'sources'

In [ ]:
for i in range(2,9):
    shapes = ogr.Open(f'{source_path}\\{i}\\shapes.shp')
    vec_layer = shapes.GetLayerByIndex(0)
    raster = gdal.Open(f'{source_path}\\{i}\\img.tif', gdal.GA_ReadOnly)
    #get relevant attributes
    ncol = raster.RasterXSize
    nrow = raster.RasterYSize
    proj = raster.GetProjectionRef()
    ext = raster.GetGeoTransform()
    #close file data to save space
    raster = None
    # Create the raster dataset
    memory_driver = gdal.GetDriverByName('GTiff')
    out_raster_ds = memory_driver.Create(f'{source_path}\\{i}\\training.tif', ncol, nrow, 1, gdal.GDT_Byte)
    # Set the ROI projection and extent to match input
    out_raster_ds.SetProjection(proj)
    out_raster_ds.SetGeoTransform(ext)
    # Fill output band with 0 blank, no class label, value
    b = out_raster_ds.GetRasterBand(1)
    b.Fill(0)
    # Rasterize the shapefile layer
    status = gdal.RasterizeLayer(out_raster_ds,  #output
                                [1],  #first band
                                vec_layer,  #referenced layer layer
                                None, None,  #No transforms
                                [0],  #burn value 0
                                ['ALL_TOUCHED=TRUE',  #rasterize all pixels touched by polygons
                                'ATTRIBUTE=id']  #match raster values to SHP ID values
                                )
    # Close dataset
    out_raster_ds = None

In [3]:
X = None
Y = None

In [4]:
for i in range(2,9):
    img_ds = gdal.Open(f'{source_path}\\{i}\\img.tif', gdal.GA_ReadOnly)
    roi_ds = gdal.Open(f'{source_path}\\{i}\\training.tif', gdal.GA_ReadOnly)

    img = np.zeros((img_ds.RasterYSize, img_ds.RasterXSize, img_ds.RasterCount),
                gdal_array.GDALTypeCodeToNumericTypeCode(img_ds.GetRasterBand(1).DataType))
    for b in range(img.shape[2]):
        img[:, :, b] = img_ds.GetRasterBand(b + 1).ReadAsArray()
    roi = roi_ds.GetRasterBand(1).ReadAsArray().astype(np.uint8)
    x = img[roi > 0, :3] #only get tagged pixels & ignore 4th band
    y = roi[roi > 0]
    if i == 2:
        X = x
        Y = y
    else:
        np.append(X, x, axis=0)
        np.append(Y, y)

In [5]:
print(X)

[[39 83 84]
 [42 84 83]
 [40 84 85]
 ...
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]]


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True)

In [7]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=50, oob_score=True)
rf = rf.fit(X_train, Y_train)
print('success')

success


In [9]:
from sklearn.metrics import accuracy_score
prediction = rf.predict(X_test)
score = accuracy_score(Y_test, prediction)
print(score)
for i in range(10):
    print(Y_test[i], prediction[i])

0.959781689289886
2 2
2 2
1 1
1 1
1 1
2 2
1 1
1 1
2 2
1 1


In [10]:
import joblib
joblib.dump(rf, "models\\RF_96.joblib")

['models\\RF_96.joblib']